In [1]:
#Dependancies 
import numpy as np 
import pandas as pd
#import seaborn as sns 
import cv2
import skimage.io as io
import keras
import os
import glob
#import matplotlib.pyplot as plt
#%matplotlib inline
#import plotly.offline as py
#import plotly.graph_objs as go
#import plotly.tools as tls 
#py.init_notebook_mode(connected=True)
import random 
import shutil

from skimage.io import imread, imshow
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

#from subprocess import check_output
#print(check_output(["ls", "~/KarlieRay/Practicum/train"]).decode("utf8"))

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from keras import backend as K
from skimage.transform import resize

Using TensorFlow backend.


In [55]:
data_dir = "/home/jray/workspace/Practicum/data"
#dimensions of images
img_width, img_height = 150, 150

In [56]:
#Split data into training and validation sets, keeping file organization. 
allFiles = glob.glob(data_dir +'/train/*/*')
print(allFiles[0])

split_point = int(round(0.7*len(allFiles)))
random.shuffle(allFiles)

train_list = allFiles[:split_point]
valid_list = allFiles[split_point:]
print('Train images: {}'.format(len(train_list)))
print('Validation images {}'. format(len(valid_list)))

# Divy them into their own directories.
train_data_path = data_dir + '/train_data'
validation_data_path = data_dir + '/valid_data'
if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
if not os.path.exists(validation_data_path):
    os.makedirs(validation_data_path)

for fpath in train_list:
    basename = fpath.split('/')[-2:]
    dest = '/'.join([train_data_path] +basename)
    if not os.path.exists('/'.join(dest.split('/')[:-1])):
        os.makedirs('/'.join(dest.split('/')[:-1]))
    shutil.copyfile(fpath, '/'.join([train_data_path] +basename))
for fpath in valid_list:
    basename = fpath.split('/')[-2:]
    dest = '/'.join([validation_data_path] +basename)
    if not os.path.exists('/'.join(dest.split('/')[:-1])):
        os.makedirs('/'.join(dest.split('/')[:-1]))    
    shutil.copyfile(fpath, '/'.join([validation_data_path] +basename))

/home/jray/workspace/Practicum/data/train/Type_1/0.jpg
Train images: 1037
Validation images 444


In [57]:
K.set_image_dim_ordering('tf') #use Tensorflow convention when inputing dat

# Create numpy array
train_image_list = list()
train_label_list = list()
for image in train_list:
    im = cv2.imread(image)
    im = resize(im, (img_width, img_height), mode='reflect').astype('float32')
    im_type = image.split('/')[-2]
    train_label_list.append(im_type.split('Type_')[1])
    train_image_list.append(im)
print(train_label_list)
print(train_image_list)

# Reshape
#train_list = train_list.reshape(train_list[0], 100, 100, 3).astype('float32')
#valid_list = Reshape(valid_list[0], 100, 100, 3).astype('float32')

['2', '2', '2', '2', '2', '2', '3', '1', '1', '2', '3', '2', '1', '2', '2', '2', '1', '2', '1', '1', '3', '2', '2', '2', '2', '3', '2', '2', '2', '2', '3', '2', '2', '2', '3', '1', '1', '2', '3', '1', '3', '2', '3', '3', '3', '2', '2', '2', '1', '2', '2', '3', '3', '2', '3', '1', '3', '2', '2', '2', '1', '2', '1', '2', '3', '3', '2', '1', '2', '2', '3', '2', '1', '3', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '2', '3', '1', '2', '2', '2', '2', '3', '2', '3', '2', '2', '2', '3', '2', '2', '2', '3', '2', '1', '2', '2', '3', '3', '3', '2', '2', '3', '2', '2', '2', '1', '3', '3', '3', '2', '3', '1', '2', '1', '3', '1', '2', '3', '2', '3', '2', '2', '2', '2', '1', '2', '3', '3', '3', '3', '1', '3', '3', '1', '1', '2', '2', '2', '2', '1', '2', '2', '2', '2', '2', '2', '2', '3', '2', '3', '3', '2', '2', '3', '3', '3', '2', '2', '2', '1', '2', '1', '3', '3', '2', '1', '3', '1', '1', '3', '2', '3', '2', '1', '2', '3', '2', '2', '2', '2', '3', '2', '2', '2', '1', '1', '3', '3', '3', '2',

In [58]:
test_image_list = list()
test_label_list = list()
for image in valid_list:
    im = cv2.imread(image)
    im = resize(im, (img_width, img_height), mode='reflect').astype('float32')
    im_type = image.split('/')[-2]
    test_label_list.append(im_type.split('Type_')[1])
    test_image_list.append(im)
print(test_label_list)
print(test_image_list)

['3', '2', '2', '3', '2', '2', '1', '2', '2', '2', '2', '2', '2', '3', '1', '2', '2', '3', '3', '1', '1', '2', '1', '3', '2', '2', '1', '2', '3', '1', '2', '2', '1', '3', '2', '3', '3', '2', '1', '2', '3', '1', '2', '3', '2', '2', '1', '2', '2', '2', '3', '2', '2', '1', '2', '2', '3', '2', '3', '2', '1', '2', '2', '3', '2', '2', '2', '3', '2', '1', '3', '2', '1', '3', '3', '1', '2', '2', '3', '1', '2', '2', '3', '2', '2', '3', '3', '1', '3', '2', '2', '2', '3', '2', '1', '3', '3', '2', '1', '2', '2', '3', '3', '2', '1', '3', '2', '2', '2', '2', '3', '2', '2', '2', '1', '3', '3', '3', '2', '2', '3', '2', '1', '3', '2', '3', '3', '2', '2', '2', '2', '2', '3', '2', '1', '3', '1', '2', '2', '2', '3', '3', '1', '1', '2', '2', '3', '3', '2', '3', '2', '1', '3', '2', '2', '3', '2', '3', '1', '3', '2', '2', '3', '1', '2', '2', '2', '3', '3', '3', '2', '3', '1', '2', '2', '1', '2', '3', '3', '3', '1', '2', '3', '3', '3', '2', '2', '1', '3', '2', '2', '2', '2', '1', '1', '2', '3', '3', '1', '2',

In [59]:
print(im.shape)

(150, 150, 3)


In [60]:
X_train = np.array(train_image_list)
Y_train = np.array(train_label_list)

#one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
num_classes = Y_train.shape[1]

In [61]:
X_test = np.array(test_image_list)
Y_test = np.array(test_label_list)

#one hot encode outputs
Y_test = np_utils.to_categorical(Y_test)


In [62]:
# Simple CNN model.
# Code modified from libphy https://github.com/libphy/dldev/blob/master/MNIST-Keras.ipynb

def simple_cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(Y_train.shape[1], activation = 'softmax'))
    
# Compile model
# Code modified from libphy https://github.com/libphy/dldev/blob/master/MNIST-Keras.ipynb

    model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy']) 
    return model

# Build model
model = simple_cnn_model()

In [63]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1037, 150, 150, 3)
(1037, 4)
(444, 150, 150, 3)
(444, 4)


In [64]:
# Fit the model
model.fit(X_train, Y_train, batch_size = 32, epochs = 5, verbose = 1)

#Final evaluation of the model
#scores = model.evaluate(train_data_path, validation_data_path, verbose = 0)
scores = model.evaluate(X_test, Y_test, verbose = 0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/5
1037/1037 [==============================] - 20s - loss: 3.5118 - acc: 0.4590    
Epoch 2/5
1037/1037 [==============================] - 19s - loss: 0.9840 - acc: 0.5429    
Epoch 3/5
1037/1037 [==============================] - 19s - loss: 0.8517 - acc: 0.6133    
Epoch 4/5
1037/1037 [==============================] - 19s - loss: 0.7405 - acc: 0.6905    
Epoch 5/5
1037/1037 [==============================] - 19s - loss: 0.6390 - acc: 0.7348    
CNN Error: 47.07%


In [65]:
# Two-layer-CNN for the Cervix dataset

# create model
def bilayer_model():
    model = Sequential()
    model.add(Convolution2D(30, (5, 3), input_shape=(img_width, img_height,3), activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, (3, 3), activation= 'relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation= 'relu' ))
    model.add(Dense(50, activation= 'relu' ))
    model.add(Dense(num_classes, activation= 'softmax' ))
    # Compile model
    model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    return model
# build model
model = bilayer_model()
# Fit model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=200,
verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 1037 samples, validate on 444 samples
Epoch 1/10
22s - loss: 1.1806 - acc: 0.4050 - val_loss: 1.0527 - val_acc: 0.3829
Epoch 2/10
21s - loss: 1.0092 - acc: 0.5236 - val_loss: 0.9815 - val_acc: 0.5113
Epoch 3/10
21s - loss: 0.9457 - acc: 0.5371 - val_loss: 0.9398 - val_acc: 0.5405
Epoch 4/10
21s - loss: 0.9047 - acc: 0.5689 - val_loss: 0.9537 - val_acc: 0.5360
Epoch 5/10
21s - loss: 0.8665 - acc: 0.5680 - val_loss: 0.9667 - val_acc: 0.5248
Epoch 6/10
21s - loss: 0.8389 - acc: 0.5959 - val_loss: 0.9408 - val_acc: 0.5608
Epoch 7/10
21s - loss: 0.8046 - acc: 0.6172 - val_loss: 0.9940 - val_acc: 0.5518
Epoch 8/10
21s - loss: 0.7850 - acc: 0.6461 - val_loss: 0.9596 - val_acc: 0.5495
Epoch 9/10
21s - loss: 0.7503 - acc: 0.6760 - val_loss: 0.9740 - val_acc: 0.5405
Epoch 10/10
21s - loss: 0.7342 - acc: 0.6856 - val_loss: 1.0282 - val_acc: 0.5405
Large CNN Error: 45.95%


In [66]:
# More convolution layers

def multilayer_cnn_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(16, (3, 3), input_shape=(img_width, img_height,3), activation= 'relu', padding='same'))
    model.add(Convolution2D(16, (3, 3), activation= 'relu', padding='valid'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(32, (3, 3), activation= 'relu', padding='same'))
    model.add(Convolution2D(32, (3, 3), activation= 'relu', padding='valid'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation= 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation= 'relu'))
    model.add(Dense(num_classes, activation= 'softmax'))
    # Compile model
    model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    return model
# build the model
model = multilayer_cnn_model()

# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=15, batch_size=200,
verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 1037 samples, validate on 444 samples
Epoch 1/15
38s - loss: 1.1697 - acc: 0.3954 - val_loss: 1.1042 - val_acc: 0.5135
Epoch 2/15
36s - loss: 1.0688 - acc: 0.5149 - val_loss: 0.9953 - val_acc: 0.5248
Epoch 3/15
36s - loss: 0.9796 - acc: 0.5458 - val_loss: 0.9696 - val_acc: 0.5113
Epoch 4/15
37s - loss: 0.9634 - acc: 0.5323 - val_loss: 0.9499 - val_acc: 0.5541
Epoch 5/15
36s - loss: 0.9379 - acc: 0.5593 - val_loss: 0.9644 - val_acc: 0.5338
Epoch 6/15
36s - loss: 0.9112 - acc: 0.5728 - val_loss: 0.9350 - val_acc: 0.5360
Epoch 7/15
36s - loss: 0.8789 - acc: 0.5796 - val_loss: 0.9247 - val_acc: 0.5360
Epoch 8/15
36s - loss: 0.8579 - acc: 0.6017 - val_loss: 0.9243 - val_acc: 0.5405
Epoch 9/15
36s - loss: 0.8354 - acc: 0.6095 - val_loss: 0.9418 - val_acc: 0.5563
Epoch 10/15
35s - loss: 0.8124 - acc: 0.6278 - val_loss: 0.9635 - val_acc: 0.5315
Epoch 11/15
36s - loss: 0.8291 - acc: 0.6114 - val_loss: 0.9522 - val_acc: 0.5270
Epoch 12/15
36s - loss: 0.7839 - acc: 0.6471 - val_loss: 0.9